In [5]:
import scrapy
from scrapper.py import gameScrapSpider
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
test = process.start()

ModuleNotFoundError: No module named 'scrapper.py'; 'scrapper' is not a package